In [1]:
# データセットをロード
import numpy as np

def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)

root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [2]:
# モデルを構築
from tensorflow import keras
from tensorflow.keras import layers

In [12]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [13]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [14]:
# トレーニングと評価
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 500, 1)]     0           []                               
                                                                                                  
 multi_head_attention (MultiHea  (None, 500, 1)      7169        ['input_5[0][0]',                
 dAttention)                                                      'input_5[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 500, 1)       0           ['multi_head_attention[0][0]']   
                                                                                                  
 layer_normalization (LayerNorm  (None, 500, 1)      2           ['dropout[0][0]']            

 mbda)                                                            'tf.__operators__.add_5[0][0]'] 
                                                                                                  
 conv1d_6 (Conv1D)              (None, 500, 4)       8           ['tf.__operators__.add_6[0][0]'] 
                                                                                                  
 dropout_7 (Dropout)            (None, 500, 4)       0           ['conv1d_6[0][0]']               
                                                                                                  
 conv1d_7 (Conv1D)              (None, 500, 1)       5           ['dropout_7[0][0]']              
                                                                                                  
 layer_normalization_7 (LayerNo  (None, 500, 1)      2           ['conv1d_7[0][0]']               
 rmalization)                                                                                     
          

2022-06-03 19:23:00.520465: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


45/45 [==============================] - 628s 14s/step - loss: 1.0776 - sparse_categorical_accuracy: 0.5038 - val_loss: 0.7563 - val_sparse_categorical_accuracy: 0.5978
Epoch 2/200
45/45 [==============================] - 638s 14s/step - loss: 0.8924 - sparse_categorical_accuracy: 0.5649 - val_loss: 0.6850 - val_sparse_categorical_accuracy: 0.6186
Epoch 3/200
45/45 [==============================] - 620s 14s/step - loss: 0.8280 - sparse_categorical_accuracy: 0.5889 - val_loss: 0.6486 - val_sparse_categorical_accuracy: 0.6505
Epoch 4/200
45/45 [==============================] - 629s 14s/step - loss: 0.7775 - sparse_categorical_accuracy: 0.6187 - val_loss: 0.6166 - val_sparse_categorical_accuracy: 0.6755
Epoch 5/200
 9/45 [=====>........................] - ETA: 8:08 - loss: 0.7468 - sparse_categorical_accuracy: 0.6163

KeyboardInterrupt: 